In [1]:
import yfinance as yf
import pandas as pd


seleciona os codigos do indice amplo   

In [2]:
url='https://raw.githubusercontent.com/BDonadelli/Codigos-em-financas/main/data/Cart_IBrA.csv'
ibra = pd.read_csv(url, sep =';' , encoding='latin-1' , decimal=',' , thousands='.' , 
                   skiprows=[0],skipfooter=2,engine='python', index_col=False)
tickers = list(ibra['Código'].values)
tickers.sort()
tickers_string = ' '.join([ticker + '.SA' for ticker in tickers])
codigos = yf.Tickers(tickers_string)


coleta dados anuais

In [3]:
df_data = pd.concat([codigos.tickers[tickers[0]+'.SA'].financials,
                     codigos.tickers[tickers[0]+'.SA'].cashflow,
                     codigos.tickers[tickers[0]+'.SA'].balance_sheet])

In [4]:
df_data = pd.concat([codigos.tickers[tickers[0]+'.SA'].financials,
                     codigos.tickers[tickers[0]+'.SA'].cashflow,
                     codigos.tickers[tickers[0]+'.SA'].balance_sheet])
colunas = [(tickers[0][:5],str(df_data.columns[i].year)) for i in range(len(df_data.columns))]
df_data.columns=pd.MultiIndex.from_tuples(colunas)

for papel in tickers[1:] :
    df_aux = pd.concat([codigos.tickers[papel+'.SA'].financials,
                        codigos.tickers[papel+'.SA'].cashflow,
                        codigos.tickers[papel+'.SA'].balance_sheet])
    # print(papel)
    colunas = [(papel[:5],str(df_aux.columns[i].year)) for i in range(len(df_aux.columns))]
    df_aux.columns=pd.MultiIndex.from_tuples(colunas)  

    df_data = pd.merge(df_data, df_aux, left_index=True, right_index=True)

    # df_aux = None

In [5]:
df_data.to_parquet("data/yf_financial_data_Y.parquet")

In [6]:
# import pyarrow.parquet as pq
# # Abrir o arquivo Parquet
# arquivo_parquet = pq.ParquetFile("data/teste.parquet")
# tabela_parquet = arquivo_parquet.read()
# df = tabela_parquet.to_pandas()

coleta dados trimestrais

In [7]:
df_data = pd.concat([codigos.tickers[tickers[0]+'.SA'].quarterly_financials,
                     codigos.tickers[tickers[0]+'.SA'].quarterly_cashflow,
                     codigos.tickers[tickers[0]+'.SA'].quarterly_balance_sheet])

colunas = [(tickers[0][:5],str(df_data.columns[i].month)+'/'+str(df_data.columns[i].year)) for i in range(len(df_data.columns))]
df_data.columns=pd.MultiIndex.from_tuples(colunas)

for papel in tickers[1:] :
    df_aux = pd.concat([codigos.tickers[papel+'.SA'].financials,
                        codigos.tickers[papel+'.SA'].cashflow,
                        codigos.tickers[papel+'.SA'].balance_sheet])
    # print(papel)
    colunas = [(papel[:5],str(df_aux.columns[i].month)+'/'+str(df_aux.columns[i].year)) for i in range(len(df_aux.columns))]
    df_aux.columns=pd.MultiIndex.from_tuples(colunas)  

    df_data = pd.merge(df_data, df_aux, left_index=True, right_index=True)


In [8]:
df_data.to_parquet("data/yf_financial_data_Q.parquet")